## Estadísticas e indicadores educativos para Sonora

Los datos se obtuvieron del sitio web https://planeacion.sep.gob.mx/entidadfederativa.aspx el 11 de Abril del 2022.


* Estadística educativa



In [1]:
#Importando librerias necesarias
import pandas as pd
import pathlib
import numpy as np

#Todo lo relacionado a graficar
import plotly.express as px

### Lectura de datos

In [2]:
#Definiendo el Path del archivo
path = pathlib.Path('..', 'data/raw/data_dir/estadistica_e_indicadores_educativos_26SON.xlsx')

In [3]:
#Leyendo los datos
data_escolar = pd.read_excel(path, sheet_name='SON', header = 10, skipfooter=2)

In [4]:
#Confirmamos que los datos se leyeron correctamente
data_escolar.head()

,Nivel educativo/Indicador,2019-2020,2020-2021,Unnamed: 3,2021-2022e/,Unnamed: 5
0,NaN,%,%,Nacional %,%,Nacional %
1,Educación básica,NaN,NaN,NaN,NaN,NaN
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268
4,Educación preescolar,NaN,NaN,NaN,NaN,NaN


### Limpieza de Datos

In [5]:
data_escolar.columns

Index(['Nivel educativo/Indicador', '2019-2020', '2020-2021', 'Unnamed: 3',
       '2021-2022e/', 'Unnamed: 5'],
      dtype='object')

In [6]:
#Primero arreglaremos los nombres de columnas
data_escolar = data_escolar.rename({'Nivel educativo/Indicador': 'Indicador',
                    '2019-2020' : '2019_2020',
                    '2020-2021': '2020_2021',
                    'Unnamed: 3': '2020_2021_nacional',
                    '2021-2022e/': '2021_2022',
                    'Unnamed: 5':'2021_2022_nacional'},axis=1)
data_escolar.head()

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional
0,NaN,%,%,Nacional %,%,Nacional %
1,Educación básica,NaN,NaN,NaN,NaN,NaN
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268
4,Educación preescolar,NaN,NaN,NaN,NaN,NaN


In [7]:
#AHora ya podemos remover el primer registro porque tiene informacion redundante
data_escolar.drop(0,axis=0,inplace=True)
data_escolar.head()

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional
1,Educación básica,NaN,NaN,NaN,NaN,NaN
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268
4,Educación preescolar,NaN,NaN,NaN,NaN,NaN
5,Atención de 3 años1/,28.15435,22.941829,40.897886,23.784795,41.65429


In [8]:
#Ahora crearemos una nueva columna con el nivel de escolaridad para cada uno de los indicadores
nivel_escolaridad = None
for index, row in data_escolar.iterrows():
    if pd.isna(row['2019_2020']):
        nivel_escolaridad = row['Indicador']
        data_escolar.loc[index, 'nivel_escolaridad'] = nivel_escolaridad
    else:
        data_escolar.loc[index, 'nivel_escolaridad'] = nivel_escolaridad

data_escolar.head()

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional,nivel_escolaridad
1,Educación básica,NaN,NaN,NaN,NaN,NaN,Educación básica
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771,Educación básica
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268,Educación básica
4,Educación preescolar,NaN,NaN,NaN,NaN,NaN,Educación preescolar
5,Atención de 3 años1/,28.15435,22.941829,40.897886,23.784795,41.65429,Educación preescolar


In [9]:
data_escolar.nivel_escolaridad.unique()

array(['Educación básica', 'Educación preescolar', 'Educación primaria',
       'Educación secundaria', 'Educación media superior',
       'Educación superior', 'Otros indicadores'], dtype=object)

In [10]:
#Como solo nos interesa los datos de menores o iguales que 14 filtraremos por nivel de escolaridad
data_escolar = data_escolar[data_escolar['nivel_escolaridad'].isin(['Educación básica', 'Educación preescolar', 'Educación primaria', 'Educación secundaria'])].copy()
data_escolar

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional,nivel_escolaridad
1,Educación básica,NaN,NaN,NaN,NaN,NaN,Educación básica
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771,Educación básica
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268,Educación básica
4,Educación preescolar,NaN,NaN,NaN,NaN,NaN,Educación preescolar
5,Atención de 3 años1/,28.15435,22.941829,40.897886,23.784795,41.65429,Educación preescolar
6,Atención de 4 años1/,82.278677,71.490893,80.624743,72.346016,81.20968,Educación preescolar
7,Atención de 5 años1/,77.417681,74.057487,75.094608,73.956523,75.419292,Educación preescolar
8,"Atención de 3, 4 y 5 años (Tasa neta de escola...",62.806387,56.375712,65.599139,56.839821,66.166459,Educación preescolar
9,Cobertura (3 a 5 años de edad)1/,63.559875,57.00468,65.908337,57.473698,66.479764,Educación preescolar
10,Educación primaria,NaN,NaN,NaN,NaN,NaN,Educación primaria


In [11]:
#Ya que tenemos la escolaridad en una columna, removeremos todos los registros que tengan valores perdidos
data_escolar.dropna(inplace=True)
data_escolar

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional,nivel_escolaridad
2,Cobertura (3 a 14 años de edad)1/,88.077762,86.496115,92.022565,86.472379,91.701771,Educación básica
3,Tasa neta de escolarización (3 a 14 años de ed...,87.477162,85.800633,91.205153,85.777163,90.887268,Educación básica
5,Atención de 3 años1/,28.15435,22.941829,40.897886,23.784795,41.65429,Educación preescolar
6,Atención de 4 años1/,82.278677,71.490893,80.624743,72.346016,81.20968,Educación preescolar
7,Atención de 5 años1/,77.417681,74.057487,75.094608,73.956523,75.419292,Educación preescolar
8,"Atención de 3, 4 y 5 años (Tasa neta de escola...",62.806387,56.375712,65.599139,56.839821,66.166459,Educación preescolar
9,Cobertura (3 a 5 años de edad)1/,63.559875,57.00468,65.908337,57.473698,66.479764,Educación preescolar
11,Abandono escolarp/,0.281927,0.226717,0.392441,0.198112,0.382257,Educación primaria
12,Reprobaciónp/,0.220672,0.215762,0.620267,0.210661,0.60832,Educación primaria
13,Eficiencia terminalp/,96.498849,97.359793,96.891692,97.645166,97.358666,Educación primaria


In [12]:
# Debido a que el indicador tiene caracteres de escape (como 1/) al final, los removeremos 
mapping = { '1/':'', 'p/':'', '4/':'', '2/':''}
for k, v in mapping.items():
    data_escolar['Indicador'] = data_escolar.Indicador.str.replace(k, v)
data_escolar.head()

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional,nivel_escolaridad
2,Cobertura (3 a 14 años de edad),88.077762,86.496115,92.022565,86.472379,91.701771,Educación básica
3,Tasa neta de escolarización (3 a 14 años de edad),87.477162,85.800633,91.205153,85.777163,90.887268,Educación básica
5,Atención de 3 años,28.15435,22.941829,40.897886,23.784795,41.65429,Educación preescolar
6,Atención de 4 años,82.278677,71.490893,80.624743,72.346016,81.20968,Educación preescolar
7,Atención de 5 años,77.417681,74.057487,75.094608,73.956523,75.419292,Educación preescolar


In [13]:
#Removemos los whitespaces que quedan al final de la columna indicador
data_escolar.Indicador = data_escolar.Indicador.str.rstrip()


In [14]:
#Ya tenemos los datos para realizar un análisis de datos exploratorio
data_escolar.head()

,Indicador,2019_2020,2020_2021,2020_2021_nacional,2021_2022,2021_2022_nacional,nivel_escolaridad
2,Cobertura (3 a 14 años de edad),88.077762,86.496115,92.022565,86.472379,91.701771,Educación básica
3,Tasa neta de escolarización (3 a 14 años de edad),87.477162,85.800633,91.205153,85.777163,90.887268,Educación básica
5,Atención de 3 años,28.15435,22.941829,40.897886,23.784795,41.65429,Educación preescolar
6,Atención de 4 años,82.278677,71.490893,80.624743,72.346016,81.20968,Educación preescolar
7,Atención de 5 años,77.417681,74.057487,75.094608,73.956523,75.419292,Educación preescolar


### Análisis de Datos Exploratorio

In [15]:
#Primerio haremos un subset de datos Tidy con los periodos que nos interesan (no contando los nacionales)
data_escolar_subset = pd.melt(data_escolar, id_vars=['Indicador', 'nivel_escolaridad' ], value_vars=['2019_2020', '2020_2021', '2021_2022'], ignore_index=False,
                value_name = 'Porcentaje', var_name= 'Periodo')
data_escolar_subset.head()

,Indicador,nivel_escolaridad,Periodo,Porcentaje
2,Cobertura (3 a 14 años de edad),Educación básica,2019_2020,88.077762
3,Tasa neta de escolarización (3 a 14 años de edad),Educación básica,2019_2020,87.477162
5,Atención de 3 años,Educación preescolar,2019_2020,28.15435
6,Atención de 4 años,Educación preescolar,2019_2020,82.278677
7,Atención de 5 años,Educación preescolar,2019_2020,77.417681


In [16]:
data_escolar_subset

,Indicador,nivel_escolaridad,Periodo,Porcentaje
2,Cobertura (3 a 14 años de edad),Educación básica,2019_2020,88.077762
3,Tasa neta de escolarización (3 a 14 años de edad),Educación básica,2019_2020,87.477162
5,Atención de 3 años,Educación preescolar,2019_2020,28.15435
6,Atención de 4 años,Educación preescolar,2019_2020,82.278677
7,Atención de 5 años,Educación preescolar,2019_2020,77.417681
8,"Atención de 3, 4 y 5 años (Tasa neta de escola...",Educación preescolar,2019_2020,62.806387
9,Cobertura (3 a 5 años de edad),Educación preescolar,2019_2020,63.559875
11,Abandono escolar,Educación primaria,2019_2020,0.281927
12,Reprobación,Educación primaria,2019_2020,0.220672
13,Eficiencia terminal,Educación primaria,2019_2020,96.498849


#### Análisis de Educación Primaria y Secundaria

In [17]:
def plotter_primaria_secundaria(indicador, dataframe):
    '''Función que grafíca los diferentes indicadores
        Comparando entre educación primaria, arregla los labels y le agrega el titulo
    '''
    dff = dataframe[dataframe['Indicador'] == indicador]
    title = 'Comparación y evolución de ' + indicador + ' para educación Primaria y Secundaria'
    fig = px.line(dff, x="Periodo", y="Porcentaje", color='nivel_escolaridad',title=title, labels={'nivel_escolaridad':'Escolaridad'})
    fig.show()

##### Comparativa de Abandono escolar

In [18]:
plotter_primaria_secundaria('Abandono escolar', data_escolar_subset)

##### Comparativa de Reprobación

In [19]:
plotter_primaria_secundaria('Reprobación', data_escolar_subset)

##### Comparativa de Tasa de Terminación

In [20]:
plotter_primaria_secundaria('Tasa de terminación', data_escolar_subset)

##### Comparativa de eficiencia terminal

In [21]:
plotter_primaria_secundaria('Eficiencia terminal', data_escolar_subset)


#### Comparativas de todos los niveles escolares

##### Comparativa de Tasa neta de escolarización

In [22]:
dff = data_escolar_subset[data_escolar_subset['Indicador'].str.contains('Tasa neta de escolarización')]
title = 'Comparación y evolución de Tasa neta de escolarización'
fig = px.line(dff, x="Periodo", y="Porcentaje", color='nivel_escolaridad', title = title,labels={'nivel_escolaridad':'Escolaridad'})
fig.show()


##### Comparativa de Cobertura

In [23]:
dff = data_escolar_subset[data_escolar_subset['Indicador'].str.contains('Cobertura')]
title = 'Comparación y evolución de Cobertura'
fig = px.line(dff, x="Periodo", y="Porcentaje", color='nivel_escolaridad', title = title,labels={'nivel_escolaridad':'Escolaridad'})
fig.show()

##### Comparativa de atención por edad en preescolar

In [24]:
dff = data_escolar_subset[data_escolar_subset['Indicador'].str.contains('Atención')]
title = 'Comparación y evolución de Atención preescolar'
fig = px.line(dff, x="Periodo", y="Porcentaje", color='Indicador', title = title,labels={'nivel_escolaridad':'Escolaridad'})
fig.show()

### Observaciones principales

#TODO

### Guardando los datos

In [25]:
path_to_save = pathlib.Path('..', 'data_clean/datos_escolares_limpios.csv')

In [27]:
data_escolar.to_csv(path_to_save)